In [1]:
import json
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


file_path = "mental.json"
with open(file_path) as file:
    data = json.load(file)

# Extract patterns and tags
patterns = []
tags = []
responses = {}

for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])
    responses[intent['tag']] = intent['responses']

# Encode the tags
label_encoder = LabelEncoder()
encoded_tags = label_encoder.fit_transform(tags)

# Tokenize the patterns
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(patterns)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(patterns)
padded_sequences = pad_sequences(sequences, padding="post")

# Prepare input and output data
input_data = np.array(padded_sequences)
output_data = tf.keras.utils.to_categorical(encoded_tags)

# Build the model
model = Sequential([
    Embedding(input_dim=1000, output_dim=16, input_length=input_data.shape[1]),
    LSTM(32, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dense(32, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Train the model
history = model.fit(input_data, output_data, epochs=170, batch_size=8)

# Save the model and tokenizer
model.save("/kaggle/working/mentalhealth_chatbot_model_final.h5.keras")
import pickle
with open("/kaggle/working/tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle)
with open("/kaggle/working/label_encoder.pickle", "wb") as enc:
    pickle.dump(label_encoder, enc)


Epoch 1/170


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0187 - loss: 4.3706
Epoch 2/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0603 - loss: 4.0337
Epoch 3/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0399 - loss: 4.1612
Epoch 4/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0244 - loss: 4.1722
Epoch 5/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0399 - loss: 4.2042
Epoch 6/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0391 - loss: 4.2379
Epoch 7/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0367 - loss: 4.1932
Epoch 8/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0497 - loss: 4.1235
Epoch 9/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0590 - loss: 4.1354
Epoch 10/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.0576 - loss: 4.0406
Epoch 11/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0741 - loss: 4.1004
Epoch 12/170
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/mentalhealth_chatbot_model_final.h5.keras'

In [ ]:
import random
import pickle
from tensorflow.keras.models import load_model

model = load_model("mentalhealth_chatbot_model_final.h5.keras")
with open("tokenizer.pickle", "rb") as handle:
    tokenizer = pickle.load(handle)
with open("label_encoder.pickle", "rb") as enc:
    label_encoder = pickle.load(enc)

# Chat function
def chatbot_response(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=input_data.shape[1], padding="post")
    prediction = model.predict(padded)
    tag = label_encoder.inverse_transform([np.argmax(prediction)])
    return random.choice(responses[tag[0]])

# Test the chatbot
print("Chatbot is ready! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    response = chatbot_response(user_input)
    print("Bot:", response)


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Chatbot is ready! Type 'quit' to exit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
Bot: Let's discuss further why you're feeling this way.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Bot: Hello there. Glad to see you're back. What's going on in your world right now?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Bot: Good night. Sweet dreams.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Bot: Not sure I understand that.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Bot: I'm very sorry to hear that but you have so much to look forward to. Please seek help by contacting: 9152987821.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Bot: Good night. Sweet dreams.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Bot: Good afternoon. How is your day going?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Bot: Good afternoon. How is your day going?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Bot: Oh okay we're done for today then. See you later
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Bot: Happy to help!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Bot: I'm listening. Please 